## Downloading the Data

In [1]:
!pip install -q gdown

In [2]:
import gdown

# The file ID from the Google Drive link
file_id = "1NR7OEboARP_fpseIZOY0Wy8Ir1NEYfL5"
url = f"https://drive.google.com/uc?id={file_id}"

# Output filename
output = "ct_scan_data.nii.gz"

# Download the file
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1NR7OEboARP_fpseIZOY0Wy8Ir1NEYfL5
From (redirected): https://drive.google.com/uc?id=1NR7OEboARP_fpseIZOY0Wy8Ir1NEYfL5&confirm=t&uuid=f42ef666-1224-46bb-aa93-fa1514feabca
To: /content/ct_scan_data.nii.gz
100%|██████████| 26.2M/26.2M [00:00<00:00, 66.0MB/s]


'ct_scan_data.nii.gz'

## Unzip the file

In [3]:
import gzip
import shutil

# Decompress the file
with gzip.open("ct_scan_data.nii.gz", 'rb') as f_in:
    with open("ct_scan_data.nii", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


## Load the Data

In [4]:
import nibabel as nib

# Load the NIfTI file
img = nib.load("ct_scan_data.nii")

# Get the data as a NumPy array
img = img.get_fdata()

# Check the shape of the data
print(img.shape)


(512, 512, 216)


## YT Start

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border
from skimage import measure
from skimage.measure import label,regionprops
from scipy import ndimage as ndi
from scipy.ndimage import measurements, center_of_mass, binary_dilation, zoom
import plotly.graph_objects as go

import ipywidgets as widgets
from IPython.display import display
import cv2

In [6]:
img.shape

(512, 512, 216)

In [7]:
img = img.transpose(1, 0, 2)
img = np.rot90(img, k=3, axes=(1, 2))

In [8]:
# Function to update the plot
def update_slice(slice_index):

    plt.figure()
    plt.pcolormesh(img[slice_index])
    plt.colorbar()
    plt.title(f"Original Slice {slice_index}")
    plt.show()

# Create a slider
slider = widgets.IntSlider(value=0, min=200, max=500, step=1, description='Slice:')
widgets.interact(update_slice, slice_index=slider)

interactive(children=(IntSlider(value=200, description='Slice:', max=500, min=200), Output()), _dom_classes=('…

<function __main__.update_slice(slice_index)>

# Extracting only Bones

In [9]:
threshold = 120

# Set all values below threshold to 0
img_bones = np.where(img >= threshold, img, 0)

In [10]:
# Function to update the plot
def update_slice(slice_index):

    plt.figure()
    plt.pcolormesh(img_bones[slice_index])
    plt.colorbar()
    plt.title(f"Bone Only Slice {slice_index}")
    plt.show()

# Create a slider
slider = widgets.IntSlider(value=0, min=200, max=300, step=1, description='Slice:')
widgets.interact(update_slice, slice_index=slider)

interactive(children=(IntSlider(value=200, description='Slice:', max=300, min=200), Output()), _dom_classes=('…

<function __main__.update_slice(slice_index)>

# Enhance edges and Thresholding

In [11]:
from scipy.ndimage import gaussian_filter, median_filter

# Edge enhancement with unsharp masking
blurred = gaussian_filter(img_bones, sigma=2)
img_edges = img_bones + (img_bones - blurred) * 2  # Adjust multiplier for edge strength

In [12]:
# Function to update the plot
def update_slice(slice_index):

    plt.figure()
    plt.pcolormesh(img_edges[slice_index])
    plt.colorbar()
    plt.title(f"Edge Enhanced Slice {slice_index}")
    plt.show()

# Create a slider
slider = widgets.IntSlider(value=0, min=200, max=300, step=1, description='Slice:')
widgets.interact(update_slice, slice_index=slider)

interactive(children=(IntSlider(value=200, description='Slice:', max=300, min=200), Output()), _dom_classes=('…

<function __main__.update_slice(slice_index)>

In [13]:
mask = img_edges > 250

In [14]:
def update_mask_slice(slice_index):
    plt.figure()
    plt.pcolormesh(mask[slice_index])
    plt.colorbar()
    plt.title(f"Masked Slice {slice_index}")
    plt.show()

slider = widgets.IntSlider(value=0, min=200, max=300, step=1, description='Slice:')
widgets.interact(update_mask_slice, slice_index=slider)

interactive(children=(IntSlider(value=200, description='Slice:', max=300, min=200), Output()), _dom_classes=('…

<function __main__.update_mask_slice(slice_index)>

# Filtering small noise and Filling (Initial)

In [15]:
from skimage.morphology import remove_small_objects
from scipy.ndimage import binary_fill_holes

# Remove small connected components (3D-aware)
cleaned_mask = remove_small_objects(mask, min_size=2000, connectivity=1)

# First pass: fill holes in axial slices (XY planes)
filled_mask = np.zeros_like(cleaned_mask, dtype=bool)
for i in range(cleaned_mask.shape[0]):
    filled_mask[i] = binary_fill_holes(cleaned_mask[i])

# Second pass: fill holes in sagittal slices (YZ planes)
for j in range(filled_mask.shape[2]):
    filled_mask[:, :, j] = binary_fill_holes(filled_mask[:, :, j])

In [16]:
def update_mask_slice(slice_index):
    plt.figure()
    plt.pcolormesh(filled_mask[slice_index])
    plt.colorbar()
    plt.title(f"Partially Filled and Noise Free Slice {slice_index}")
    plt.show()

slider = widgets.IntSlider(value=0, min=200, max=300, step=1, description='Slice:')
widgets.interact(update_mask_slice, slice_index=slider)

interactive(children=(IntSlider(value=200, description='Slice:', max=300, min=200), Output()), _dom_classes=('…

<function __main__.update_mask_slice(slice_index)>

# Femur and Tibia Separation

In [17]:
from scipy.ndimage import label, center_of_mass

# filled_mask is a 2D boolean mask where both bones are connected regions

# Label connected components
labeled_mask, num_labels = label(filled_mask)

# Compute vertical (y-axis) centroids
centroids = center_of_mass(filled_mask, labeled_mask, range(1, num_labels + 1))
centroids_y = [c[0] for c in centroids]  # y-coordinates (row)

# Sort labels by vertical position (top = femur, bottom = tibia)
sorted_labels = [label_idx for _, label_idx in sorted(zip(centroids_y, range(1, num_labels + 1)))]

# Create masks
femur_mask = (labeled_mask == sorted_labels[0]) | (labeled_mask == sorted_labels[1])
tibia_mask = labeled_mask == sorted_labels[2]


In [18]:
def update_mask_slice(slice_index):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    im1 = axs[0].pcolormesh(tibia_mask[slice_index])
    axs[0].set_title(f"Tibia Mask Slice {slice_index}")
    plt.colorbar(im1, ax=axs[0])

    im2 = axs[1].pcolormesh(femur_mask[slice_index])
    axs[1].set_title(f"Femur Mask Slice {slice_index}")
    plt.colorbar(im2, ax=axs[1])

    plt.show()

slider = widgets.IntSlider(value=0, min=200, max=300, step=1, description='Slice:')
widgets.interact(update_mask_slice, slice_index=slider)

interactive(children=(IntSlider(value=200, description='Slice:', max=300, min=200), Output()), _dom_classes=('…

<function __main__.update_mask_slice(slice_index)>

# Filling Femur and Tibia (Individually)

In [19]:
def preprocess_and_fill(mask_bool, kernel_size=15, iterations=2):
    # Convert boolean mask to uint8 for cv2 operations (0 and 255)
    mask_uint8 = mask_bool.astype(np.uint8) * 255

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    closed = cv2.morphologyEx(mask_uint8, cv2.MORPH_CLOSE, kernel, iterations=iterations)

    # Convert back to boolean for filling holes
    closed_bool = closed.astype(bool)

    # Fill holes on the boolean mask
    filled = binary_fill_holes(closed_bool)

    return filled

# Apply separately
femur_mask_filled = preprocess_and_fill(femur_mask)
tibia_mask_filled = preprocess_and_fill(tibia_mask)

In [20]:
def update_mask_slice(slice_index):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    im1 = axs[0].pcolormesh(tibia_mask_filled[slice_index])
    axs[0].set_title(f"Tibia Filled Slice {slice_index}")
    plt.colorbar(im1, ax=axs[0])

    im2 = axs[1].pcolormesh(femur_mask_filled[slice_index])
    axs[1].set_title(f"Femur Filled Slice {slice_index}")
    plt.colorbar(im2, ax=axs[1])

    plt.show()

slider = widgets.IntSlider(value=0, min=200, max=300, step=1, description='Slice:')
widgets.interact(update_mask_slice, slice_index=slider)

interactive(children=(IntSlider(value=200, description='Slice:', max=300, min=200), Output()), _dom_classes=('…

<function __main__.update_mask_slice(slice_index)>

# Masking Orignal and Saving

In [21]:
print(femur_mask_filled.shape)
print(tibia_mask_filled.shape)
print(img.shape)

(512, 216, 512)
(512, 216, 512)
(512, 216, 512)


In [22]:
femur_segment = img * femur_mask_filled
tibia_segment = img * tibia_mask_filled
overall_mask = femur_mask_filled | tibia_mask_filled
overall_segment = img * overall_mask

In [23]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def update_mask_slice(slice_index):
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    im1 = axs[0].pcolormesh(femur_segment[slice_index])
    axs[0].set_title(f"Femur Segment Slice {slice_index}")
    plt.colorbar(im1, ax=axs[0])

    im2 = axs[1].pcolormesh(tibia_segment[slice_index])
    axs[1].set_title(f"Tibia Segment Slice {slice_index}")
    plt.colorbar(im2, ax=axs[1])

    im3 = axs[2].pcolormesh(overall_segment[slice_index])
    axs[2].set_title(f"Overall Segment Slice {slice_index}")
    plt.colorbar(im3, ax=axs[2])

    plt.tight_layout()
    plt.show()

slider = widgets.IntSlider(value=0, min=200, max=300, step=1, description='Slice:')
widgets.interact(update_mask_slice, slice_index=slider)


interactive(children=(IntSlider(value=200, description='Slice:', max=300, min=200), Output()), _dom_classes=('…

<function __main__.update_mask_slice(slice_index)>

# Saving the Original Mask

In [24]:
overall_segment.shape

(512, 216, 512)

In [25]:
# Step 1: Reverse the rotation
overall_segment = np.rot90(overall_segment, k=1, axes=(1, 2))

# Step 2: Reverse the transpose
overall_segment = overall_segment.transpose(1, 0, 2)

In [26]:
overall_segment.shape

(512, 512, 216)

In [27]:
# Convert to float32
overall_segment = overall_segment.astype(np.float32)

# Create a NIfTI image
nifti_img = nib.Nifti1Image(overall_segment, affine=np.eye(4))

# Save to file
nib.save(nifti_img, 'overall_segment.nii.gz')

# Contour Expansion

In [28]:
from scipy.ndimage import binary_dilation
from scipy.ndimage import generate_binary_structure

def expand_mask(mask, spacing, expansion_mm=2.0):

    # Calculate how many voxels correspond to the desired expansion
    radius_voxels = [int(np.ceil(expansion_mm / s)) for s in spacing]

    # Create an ellipsoidal structuring element matching the voxel dimensions
    zz, yy, xx = np.ogrid[
        -radius_voxels[0]:radius_voxels[0]+1,
        -radius_voxels[1]:radius_voxels[1]+1,
        -radius_voxels[2]:radius_voxels[2]+1
    ]

    struct_elem = ((zz * spacing[0])**2 +
                   (yy * spacing[1])**2 +
                   (xx * spacing[2])**2) <= expansion_mm**2

    # Apply binary dilation using the custom structuring element
    expanded_mask = binary_dilation(mask, structure=struct_elem)

    return expanded_mask.astype(mask.dtype)

In [29]:
# Load original segment to get affine and spacing
img_nii = nib.load("overall_segment.nii.gz")
original_mask = img_nii.get_fdata()
affine = img_nii.affine
spacing = img_nii.header.get_zooms()[:3]  # (z, y, x)

# Create expanded masks
expanded_mask_2mm = expand_mask(original_mask > 0, spacing, expansion_mm=2.0)
expanded_mask_4mm = expand_mask(original_mask > 0, spacing, expansion_mm=4.0)

In [30]:
# Convert the boolean masks to integer
expanded_mask_2mm = expanded_mask_2mm.astype(np.float32)
expanded_mask_4mm = expanded_mask_4mm.astype(np.float32)

# Save as NIfTI files
nib.save(nib.Nifti1Image(expanded_mask_2mm, affine), "overall_segment_expanded_2mm.nii.gz")
nib.save(nib.Nifti1Image(expanded_mask_4mm, affine), "overall_segment_expanded_4mm.nii.gz")

# Randomized Contour Adjustment

In [31]:
def randomize_expanded_mask(original_mask, expanded_mask_2mm, random_fraction=0.5):

    # Ensure inputs are boolean
    original_mask = original_mask.astype(bool)
    expanded_mask_2mm = expanded_mask_2mm.astype(bool)

    # Define the ring: voxels in expanded_mask_2mm but not in original_mask
    ring = expanded_mask_2mm & (~original_mask)
    ring_indices = np.argwhere(ring)

    # Number of voxels to randomly pick based on fraction
    n_pick = int(len(ring_indices) * random_fraction)

    # Randomly select voxels from ring
    if n_pick > 0:
        chosen_indices = ring_indices[np.random.choice(len(ring_indices), size=n_pick, replace=False)]
    else:
        chosen_indices = np.empty((0, 3), dtype=int)

    # Create randomized mask: start with original mask
    randomized_mask = np.copy(original_mask)

    # Add randomly chosen voxels
    for idx in chosen_indices:
        randomized_mask[tuple(idx)] = True

    return randomized_mask.astype(np.uint8)


In [32]:
# Create randomized masks with different random fractions
random_fraction_1 = 0.5  # 50% random expansion
random_fraction_2 = 0.8  # 80% random expansion

orginal_randomized_1 = randomize_expanded_mask(original_mask > 0, expanded_mask_2mm > 0, random_fraction=random_fraction_1)
orginal_randomized_2 = randomize_expanded_mask(original_mask > 0, expanded_mask_2mm > 0 , random_fraction=random_fraction_2)


In [33]:
nib.save(nib.Nifti1Image(orginal_randomized_1, affine), "orginal_randomized_mask_1.nii.gz")
nib.save(nib.Nifti1Image(orginal_randomized_2, affine), "orginal_randomized_mask_2.nii.gz")

# Medial and Lateral Lowest Points

In [34]:
def find_medial_lateral_lowest_points(mask):
    z_dim, y_dim, x_dim = mask.shape
    print("Mask shape (Z, Y, X):", mask.shape)

    mid_x = x_dim // 2
    padded = np.pad(mask, pad_width=1, mode='constant', constant_values=0)
    surface_voxels = []

    for z in range(z_dim):
        for y in range(y_dim):
            for x in range(x_dim):
                if mask[z, y, x]:
                    neighbors = [
                        padded[z+1, y+1, x+2],  # x+1
                        padded[z+1, y+1, x],    # x-1
                        padded[z+1, y+2, x+1],  # y+1
                        padded[z+1, y, x+1],    # y-1
                        padded[z+2, y+1, x+1],  # z+1
                        padded[z, y+1, x+1],    # z-1
                    ]
                    if any(n == 0 for n in neighbors):
                        surface_voxels.append((z, y, x))

    surface_voxels = np.array(surface_voxels)
    print("Total surface voxels found:", len(surface_voxels))

    medial_surface = surface_voxels[surface_voxels[:, 2] < mid_x]
    lateral_surface = surface_voxels[surface_voxels[:, 2] >= mid_x]
    print("Medial surface voxels:", len(medial_surface))
    print("Lateral surface voxels:", len(lateral_surface))

    if len(medial_surface) == 0 or len(lateral_surface) == 0:
        raise ValueError("Could not find medial or lateral surface points. Check your mask orientation or content.")

    medial_lowest_point = tuple(medial_surface[np.argmin(medial_surface[:, 0])])
    lateral_lowest_point = tuple(lateral_surface[np.argmin(lateral_surface[:, 0])])

    return medial_lowest_point, lateral_lowest_point

In [35]:
# Step 1: Reverse the rotation
tibia_segment = np.rot90(tibia_segment, k=1, axes=(1, 2))

# Step 2: Reverse the transpose
tibia_segment = tibia_segment.transpose(1, 0, 2)

In [36]:
tibia_segment = tibia_segment.astype(np.float32)

# Create a NIfTI image
nifti_img = nib.Nifti1Image(tibia_segment, affine=np.eye(4))

# Save to file
nib.save(nifti_img, 'tibia_segment.nii.gz')

In [37]:
# Load tibial mask
tibia_nii = nib.load("tibia_segment.nii.gz")
tibia_mask = (tibia_nii.get_fdata() > 0).astype(bool)

In [38]:
tibia_mask.shape

(512, 512, 216)

In [39]:
# Transpose from (X, Y, Z) → (Z, Y, X)
mask = np.transpose(mask, (2, 1, 0))  # Shape becomes (216, 512, 512)

In [40]:
medial_pt, lateral_pt = find_medial_lateral_lowest_points(mask)
print("Medial lowest point (z, y, x):", medial_pt)
print("Lateral lowest point (z, y, x):", lateral_pt)

Mask shape (Z, Y, X): (512, 216, 512)
Total surface voxels found: 330521
Medial surface voxels: 48785
Lateral surface voxels: 281736
Medial lowest point (z, y, x): (np.int64(72), np.int64(215), np.int64(244))
Lateral lowest point (z, y, x): (np.int64(17), np.int64(0), np.int64(349))
